In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn import svm
from sklearn.metrics import accuracy_score
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
dataset_17 = pd.read_csv('fitzpatrick17k.csv')

In [23]:
dataset_17

,skin_tone_category,label,img_id
0,dark,melanoma,18.png
1,dark,melanoma,2450.png
2,dark,melanoma,2562.png
3,dark,melanoma,5848.png
4,dark,melanoma,8250.png
5,dark,melanoma,9429.png
6,dark,melanoma,10263.png
7,dark,melanoma,10867.png
8,dark,melanoma,11319.png
9,dark,melanoma,11772.png


In [7]:
dataset_17 = dataset_17[dataset_17['fitzpatrick_scale'] != -1]
dataset_17 = dataset_17[dataset_17['label'] == "melanoma"]

In [8]:
dataset_17

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
13,b87804452f60aa162a6d29c0f66a2466,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLlmmlm...
18,4c3f795cf8eb72b946f9bd2642cf23c1,6,5,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
330,22f13657c4a6b4a3fc6ea06d313c3725,3,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
349,480fd80167d7865ce9aafe14aba4ec9d,5,-1,melanoma,malignant melanoma,malignant,NaN,NaN,httpwwwdermaamincomsiteimagesclinicalpicooralm...
488,e170d124649f3cddf0e8d2ac7a4e597f,1,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
...,...,...,...,...,...,...,...,...,...
16435,80602e43414e592443aee9e67fcca5c7,4,4,melanoma,malignant melanoma,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=4299,httpwwwatlasdermatologicocombrimgimageId4299.jpg
16454,db6590971625b0e98971f449b4ed6ee7,3,3,melanoma,malignant melanoma,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=4253,httpwwwatlasdermatologicocombrimgimageId4253.jpg
16465,9ee17127331be5b1c1f2bdea846ee60f,2,3,melanoma,malignant melanoma,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=4245,httpwwwatlasdermatologicocombrimgimageId4245.jpg
16487,30b3bb79b3d16b45ec095a91cd9c32ee,4,4,melanoma,malignant melanoma,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=4255,httpwwwatlasdermatologicocombrimgimageId4255.jpg


In [9]:
dataset_17['fitzpatrick_scale'].value_counts()

fitzpatrick_scale
2    81
3    50
1    41
4    38
5    23
6    11
Name: count, dtype: int64

In [12]:
pd.set_option('display.max_columns', None)

In [20]:
dataset_17[dataset_17['fitzpatrick_scale'].isin([1, 2])][0:12]

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
13,b87804452f60aa162a6d29c0f66a2466,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLlmmlm...
488,e170d124649f3cddf0e8d2ac7a4e597f,1,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
527,83cd9569896510c012ebc6c5bb7f3c74,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicaacrol...
578,cc63958a1cedccb7dc1f0628f010206e,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
594,576c5c37a21c1c767733e486ac69b71c,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLlmmlm...
760,9180caf457da660165757eb4de0cd35c,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
867,203f72e12bfd9132ba8f843eaa9a894f,2,1,melanoma,malignant melanoma,malignant,1 Diagnostic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
1043,163d9c550fad4704990a3d1b8183d0f8,2,2,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
1054,c05b03bcef867fca1e842f5226089b2c,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicaacrol...
1188,c12375fa8da3696e9198df1fd74703c1,1,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...


In [10]:
dataset_17[dataset_17['fitzpatrick_scale'].isin([5, 6])]

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
18,4c3f795cf8eb72b946f9bd2642cf23c1,6,5,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
349,480fd80167d7865ce9aafe14aba4ec9d,5,-1,melanoma,malignant melanoma,malignant,NaN,NaN,httpwwwdermaamincomsiteimagesclinicalpicooralm...
879,9d994f1278a9788612fa2af179328c31,6,6,melanoma,malignant melanoma,malignant,NaN,NaN,httpwwwdermaamincomsiteimagesclinicalpicooralm...
2450,d10722216af2bab5bfc450259169f9fe,6,6,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicrrecur...
2562,7d4182584c503100338e00b7b46bcc4b,5,6,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicrrecur...
5848,ac04675f4d3943dfcfadd5284c60ef20,5,4,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
8250,d4862d1b866e2c536dde339f60de4f0d,5,4,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
9429,617f9ddb778028f305520a92ac052c68,6,6,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicrrecur...
10263,0d8457afebb67905127da37baece43d8,5,4,melanoma,malignant melanoma,malignant,1 Diagnostic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...
10867,658446bde66c64e374d317e40fb40f91,6,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicaacrol...
